
# Goal and Enviroment


In this project, I chose to build a verison II (see [ReadMe](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/README.md)) reinforcement learning (RL) agent that controls a robotic arm within Unity's [Reacher](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#reacher) environment. The goal is to get 20 different robotic arms to maintain contact with the green spheres.

A reward of +0.1 is provided for each time-step that the agent's hand is in the goal location. In order to solve the environment, our agent must achieve a score of +30 averaged across all 20 agents for 100 consecutive episodes.

#### Policy-based vs Value-based Methods
There is one key difference in the Reacher environment compared to the previous ['Navigation' project](https://github.com/readerwei/Reinforcement_Learning_Degree/tree/master/p1_navigation):

**Continuous action space** &mdash; The action space is now _continuous_, which allows each agent to execute more complex and precise movements. Essentially, there's an unlimited range of possible action values to control the robotic arm, whereas the agent in the Navigation project was limited to four _discrete_ actions: left, right, forward, backward.

Given the additional complexity of this environment, the pure **value-based method** we used for the last project is not suitable &mdash; i.e., the Deep Q-Network (DQN) algorithm. Most importantly, we need an algorithm that allows the robotic arm to utilize its full range of movement. For this, we'll need to explore a different class of algorithms called **policy-based methods**.

Here are some advantages of policy-based methods:
- **Continuous action spaces** &mdash; Policy-based methods are well-suited for continuous action spaces.
- **Stochastic policies** &mdash; Both value-based and policy-based methods can learn deterministic policies. However, policy-based methods can also learn true stochastic policies.
- **Simplicity** &mdash; Policy-based methods directly learn the optimal policy, without having to maintain a separate value function estimate. With value-based methods, the agent uses its experience with the environment to maintain an estimate of the optimal action-value function, from which an optimal policy is derived. This intermediate step requires the storage of lots of additional data since you need to account for all possible action values. Even if you discretize the action space, the number of possible actions can be quite high. For example, if we assumed only 10 degrees of freedom for both joints of our robotic arm, we'd have 1024 unique actions (2<sup>10</sup>). Using DQN to determine the action that maximizes the action-value function within a continuous or high-dimensional space requires a complex optimization process at every time step.


# Learning Algorithm

During the project I have tested two different algos on the target agent. One is Proximal Policy Optimization (PPO) Algorithm, and the other is Deep Deterministic Policy Gradient (DDPG). It turned out that DDPG performs better in this case, as can be seen from follows. 

## Deep Deterministic Policy Gradient (DDPG)
The algorithm is outlined in [this paper](https://arxiv.org/pdf/1509.02971.pdf), _Continuous Control with Deep Reinforcement Learning_, by researchers at Google Deepmind. In this paper, the authors present "a model-free, off-policy actor-critic algorithm using deep function approximators that can learn policies in high-dimensional, continuous action spaces." They highlight that DDPG can be viewed as an extension of Deep Q-learning to continuous tasks.

I used [ddpg_agent.py](https://github.com/udacity/deep-reinforcement-learning/blob/master/ddpg-pendulum/ddpg_agent.py) from Udacity's course Github as a template. I further experimented with the DDPG algorithm based on other concepts covered in Udacity's classroom and lessons. My understanding and implementation of this algorithm (including various customizations) are discussed below.

#### DQN vs DDPG
In the DQN Algorithm, one calculates the optimal state-action values for continuous states and discrete action space, then choose the optimal action according to $argmax$ operation. However, in the continuous action spaces, performing such $argmax$ is difficult. 

On the other hand, one can use a policy based method to optimize the policy gradient directly. Actor-critic methods leverage the strengths of both policy-based and value-based methods. Using a policy-based approach, the agent (actor) learns how to act by directly estimating the optimal policy and maximizing reward through gradient ascent. Meanwhile, employing a value-based approach, the agent (critic) learns how to estimate the value (i.e., the future cumulative reward) of different state-action pairs. Actor-critic methods combine these two approaches in order to accelerate the learning process. Actor-critic agents are also more stable than value-based agents, while requiring fewer training samples than policy-based agents.

DDPG attack the problem from a third direction. It is ultimately still use a QNetwork $Q$ to evaluate the advantage of a state, action pair. However, it uses another deep network $g$ to approximate the action space, similar to the actor function in the actor-critic method:
$$ g_\pi(s; \Phi) = \bf{a} $$
The goal is to find a function $\Phi$ that maximizes the value function $Q^*(s,\bf{a}; \Theta)$, where $\bf{a}$ is the output of the above actor function. Here, the Q Network will implement a one step Temporal-Difference methods (TD) to evaluate the action-values, similar to the critic agent in AC method. As in the previous project, to increase the efficiency of learning, we will adopt a Replay Buffer $\mathbb{D}$. Also, to avoid the moving target problem, we will setup target functions for both $\Phi$ and $\Theta$, which will be labeled as $\Phi^-$ and $\Theta^-$. 

Thus, the algorithm can be summarized as follows:
- Take action $\bf{a} = g_\pi(s; \Phi) + \bf{z}$ from $s$, where $\bf{z}$ is a random noise for exploration, which will be elaborated below. 
- Observe $s'$ and reward $R$, add $(s, a, R, s')$ to a Replay Buffer $\mathbb{D}$
- Sample a mini-batch of $(s^{(i)}, a^{(i)}, R^{(i)}, s'^{(i+1)}$ from $\mathbb{D}$
- Update $\Theta$ using TD(1) Advantage function:
    $$\Theta \leftarrow \Theta - \eta \nabla_{\Theta} A(s,\bf{a}; \Theta)$$ 
    where 
    $$ A = \sum_{i}{\left[R^{i}+\gamma Q(s^{(i+1)}, g_\pi(s; \Phi^-); \Theta^-) - Q(s^{(i)},\bf{a}^{(i)}; \Theta) \right]^2} $$
    
- Update $\Phi$:
    $$\Phi \leftarrow \Phi - \lambda \nabla_{\Phi} Q(s,\bf{a}; \Phi)$$
- Slowly update the target network: 
    $\Theta^- \leftarrow \tau \Theta + (1-\tau) \Theta^-$ and $\Phi^- \leftarrow \tau \Phi + (1-\tau) \Phi^-$

Please find the DDPG logic implemented as part of the `Agent()` class in [ddpg_agent.py](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L30) of the source code. The networks $A$ and $Q$ can be found via their respective `Actor()` and `Critic()` classes [model.py](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/model.py).


#### Experience Replay
Experience replay allows the RL agent to learn from past experience.

As with DQN in the previous project, DDPG also utilizes a replay buffer to gather experiences from each agent. Each experience is stored in a replay buffer as the agent interacts with the environment. In this project, there is one central replay buffer utilized by all 20 agents, therefore allowing agents to learn from each others' experiences. It turned out to be critically important to achieve a good result in short number of iterations. 

The replay buffer contains a collection of experience tuples with the state, action, reward, and next state $(s, a, R, s')$. Each agent samples from this buffer as part of the learning step. Experiences are sampled randomly, so that the data is uncorrelated. This prevents action values from oscillating or diverging catastrophically, since a naive algorithm could otherwise become biased by correlations between sequential experience tuples.

Also, experience replay improves learning through repetition. By doing multiple passes over the data, our agents have multiple opportunities to learn from a single experience tuple. This is particularly useful for state-action pairs that occur infrequently within the environment.

The implementation of the replay buffer can be found in the [ddpg_agent.py](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L189) of the source code.

Also, the size of the buffer is also very important to the success of the training. Initially I was using the default buffer size 1e5, however, it never reached the target performance with this size. Finally, I changed the size to 1e6 and reached the performances shown in this report. 

#### Exploration vs Exploitation
One challenge is choosing which action to take while the agent is still learning the optimal policy. Should the agent choose an action based on the rewards observed thus far? Or, should the agent try a new action in hopes of earning a higher reward? This is known as the **exploration vs. exploitation dilemma**.

In the Navigation project, I addressed this by implementing an [𝛆-greedy algorithm]. This algorithm allows the agent to systematically manage the exploration vs. exploitation trade-off. The agent "explores" by picking a random action with some probability epsilon `𝛜`. Meanwhile, the agent continues to "exploit" its knowledge of the environment by choosing actions based on the deterministic policy with probability (1-𝛜). However, this approach won't work for controlling a robotic arm. The reason is that the actions are no longer a discrete set of simple directions (i.e., up, down, left, right). 

Instead, we'll use the **Ornstein-Uhlenbeck process**, as suggested in the previously mentioned [paper by Google DeepMind](https://arxiv.org/pdf/1509.02971.pdf) (see bottom of page 4). The Ornstein-Uhlenbeck process adds a certain amount of noise to the action values at each timestep. This noise is correlated to previous noise, and therefore tends to stay in the same direction for longer durations without canceling itself out. This allows the arm to maintain velocity and explore the action space with more continuity.

You can find the Ornstein-Uhlenbeck process implemented  in the class of [`OUNoise`](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L167) from the source code.



# Code Implementation

The code used here is adapted from the ["ddpg-pendulum"](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) tutorial from the Deep Reinforcement Learning Nanodegree, and has been slightly adjusted for being used with the Reacher environment.

The code consist of :

- model.py : In this python file, a PyTorch Actor and Critic classes are implemented which inherits nn.Module base class. This is a regular fully connected Deep Neural Network using the [PyTorch Framework](https://pytorch.org/docs/0.4.0/). The actor network will be trained to generate the actions to perform depending on the environment observed states while the critic network will be trained to evaluate the advantage of such actions. These Neural Networks are used by the DDPG agent and is composed of :

  - Actor network
      - input layer of size equal to the state_size (33)
      - 2 hidden fully connected layers of 128 and 64 cells each
      - output layer which returns the actions to be taken by the agent, depends on the action_size parameter passed in the constructor, which is 4 in our problem
      
  - Critic network
      - input layer of size 33
      - 1 hidden fully connected layers of 128 cells with 4 extra units souring the action values
      - 2nd hidden fully connected layers of 64 cells
      - output layer which returns the Q-value of size 1
  
- ddpg_agent.py : In this python file, a DDPG agent, an OUNoise class and a Replay Buffer memory (used by the DDPG agent) are defined.

  - The Agent class is implemented, as described in the DDPG algorithm. It provides several methods :
    - constructor : 
      - Initialize the memory buffer (*Replay Buffer*)
      - Initialize the OUNoise instance
      - Initialize 2 instance of the Actor  Neural Network : the *target* network and the *local* network
      - Initialize 2 instance of the Critic Neural Network : the *target* network and the *local* network
      
    - step() : 
      - Allows to store a step taken by the agent (state, action, reward, next_state, done) in the Replay Buffer/Memory
      - Every step (and if their are enough samples available in the Replay Buffer), sample from the replay buffer and perform the learning steps. 
      
    - act():
      - It returns actions for the given state as per current policy (actor) network 
      - Add noise to each step the actor is taking
      
    - learn():
      - which update both critic and actor Neural Network value parameters by standard training procedure using given batch of experiences from the Replay Buffer
      - update the two *target* networks' weights with continuous blending from the current weight values from the *local* network
      
    - soft_update():
      - It is called by learn() to slowly blends the weights of the *local* network weights into the *target* Neural Network. 
      
  - The ReplayBuffer class implements a fixed-size buffer to store experience tuples  (state, action, reward, next_state, done) 
    - add() allows to add an experience step to the memory
    
    - sample() allows to randomly sample a minibatch of experience steps for the learning  
    
  - The OUNoise class implements Ornstein-Uhlenbeck process to serve an exploration mechanism for the action 
    - reset() allows to reset the internal state (= noise) to mean (mu)
    
    - sample() allows to randomly return a noise sample to be added to the action
    
    
- Continous-Control_ddpg_gpu.ipynb : This Jupyter notebooks allows to train the agent. More in details it allows to :
  - Import the Necessary Packages 
  - Examine the State and Action Spaces
  - Take Random Actions in the Environment
  - Train an agent using DDPG, the main function of training is called ddpg()
  - Use Tensorboard to monitor the training procedure
  - Plot the scores


# DDPG Parameters and Results

## Hyperparameters

### Agent Hyperparameters

In total, there are six hyper-parameters related to the DDPG Agent itself. The hyperparameters of the agent are listed as below:
```python
L1_SIZE = 128           # size of 1st fully connected layer in the actor and critic network
L2_SIZE = 64            # size of 2nd fully connected layer in the actor and critic network
LR_ACTOR  = 2e-3        # learning rate of the actor 
LR_CRITIC = 1e-4        # learning rate of the critic
GAMMA = 0.98            # discount factor for Q value calculation
TAU = 1e-3              # for soft update of target parameters
```
Note that in principle one can use different network structure for actor and critic network. Here, to reduce some complexity of the system I have used the same structure for both networks and the sizes listed above turned out to be enough to generate good performance. 

To reduce the variance of future steps in the TD(1) evaluation, I have reduced the discount rate from 0.99 to 0.98 which receives a good effect. 

The two learning rate was the major hyper-parameters for me to tune. In order to get a well guided direction for the tuning, I leverage the [tensorboard](https://pytorch.org/docs/stable/tensorboard.html) capability within PyTorch. Details can be found in the next subsection.



### Replay Buffer Hyperparameters
The Replay Buffer has two hyperparameters that determine the size of the buffer and sample size of the minibatch:
```python
BUFFER_SIZE = int(1e6)  # replay buffer size
BATCH_SIZE = 256        # minibatch size
```

As indicated in the previous section, the size of the buffer was tuned to have enough samples which is a critical point.

### OUNoise Hyperparameters
The Ornstein-Uhlenbeck process itself has three hyperparameters that determine the noise characteristics and magnitude:
- mu: the long-running mean
- theta: the speed of mean reversion
- sigma: the volatility parameter

Of these, I only tuned sigma and theta. After running a few experiments, I reduced sigma from 0.3 to 0.05. The reduced noise volatility seemed to help the model converge faster.

Notice also there's an epsilon parameter used to decay the noise level over time. I got this idea from [Thomas Tracey](https://github.com/tommytracey/DeepRL-P2-Continuous-Control), although I should say it is a natural adaption from the magnitude of $\epsilon$-greedy policy. This decay mechanism ensures that more noise is introduced earlier in the training process (i.e., higher exploration), and the noise decreases over time as the agent gains more experience (i.e., higher exploitation). The decay rate of OU Noise is tuned during experimentation.

You can find the epsilon process implemented [here](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L96) in the `Agent.act()` method in `ddpg_agent.py` of the source code. While the epsilon decay is performed [here](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L151) as part of the learning step.

The final noise parameters were set as follows:

```python
Noise_SIGMA = 0.05         # Ornstein-Uhlenbeck noise vol parameter
NOISE_THETA = 0.1          # Ornstein-Uhlenbeck noise speed parameter
EPSILON_DECAY = 2e-6       # decay rate for noise process
```

## Model Tuning and Monitoring

As is well known in the AI community, all the AI programs would experience two stage of debugging: code debug and hyper-parameters debug. In terms of code debugging, it is relatively easy and straight forward, although time consuming, there are well developed tools and general practices to follow. I have been using the [`pdb`](https://docs.python.org/3/library/pdb.html) package to perform the code debugging.

However, hyper-parameter tunning/debugging is a much more complicated process. As counted from above, we have 11 hyper-parameters to tune in this project and not each and everyone had the same effect on the final outcome. And initially one has no clue of what is a more important parameter to tune. Learned from Adam Green's [blog](https://adgefficiency.com/dqn-debugging/) on tuning the DQN network, I realized that utilizing Tensorboard to monitor the training process would have immense value for gaining insights about the parameters. Thus, I added the following code in the training process (Cell [12] of the [notebook](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/Continuous_Control_ddpg_gpu.ipynb)) to launch the Tensorboard environment:
```python
%tensorboard --logdir runs/ddpg_v2 --port 6009
```
To facilitate the process, I added a step counter and Tensorboard SummaryWriter into the class variable of the Agent class, which is indicated in this line of [codes](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L45) in the `ddpg_agent.py`. I monitor the detail actions of the agent took to see what kind action spaces have been covered, as can be seen [here](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L75). Also, I have noted down the loss function of the actor and the critic network to see how the losses are minimized during the training, as shown [here](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ddpg_agent.py#L129). Initially, I was using the same learning rate for the actor and the critic. However, after noticing that the critic loss function has some large swings, I lowered the learning rate for the critic and thus achieved much better and stable training. The detail tuning processes facilitated by those metrics will be discussed below. First of all, let us have a look at what the Tensorboard is showing after launching.

![Tensorboard Overview](./images/tensorflow_all.png)

One can see that different training iterations are put into different folder and shown in different colors. The yellow line shows the final tuned agent's result, while the pink one shows the rewards gained before tuning hyper-parameters. Note that the horizontal axis is showing steps rather than episodes. In each episodes there are 1000 steps, thus 100k steps indicate that the agent is at 100th episode.

### Reward Comparison

The score plots show that pre-tuned agent significantly under performs the yellow one, and cannot reach average of 30 even trained for longer steps. 
![Tensorboard Overview](./images/score_compare.png)

### Action Space Exploration

The first thing I looked at is whether the agent explores enough action spaces by plotting the actions taken by the agent each step:

![Plot of Actions](./images/action_1.png)

One can see the pink line which represents the pre-tuned agent (pink) only covers small action spaces around '-1', which is not a good sign of enough exploration. In contrast, one can see that the well-tuned agent (yellow) explores all the action spaces from -1 to 1. 

### Noise Volatility Tuning

Next to tune is the sigma of the OU noise. As mentioned above, I tuned it down to 0.05. The rational can be read from the plots below.

![Plot of noises](./images/action_noise.png)

The mean actions output from the actor is between -0.1 to 0.1 (upper figure), thus the new sigma (equal to 0.05) will bound the noise amplitude to below 0.03 (lower figure), which is 30% of the signal. Before that, noise sigma was at a level too big which leads to a large noise/signal ratio. In that case, the training led to too much exploration. 

##### Correction after first submission
In fact in the previous submission, the OU noise were coded incorrectly. A typical OU noise is driven by Brownian motion, which is usually standard normal distribution. I have used `random.random`, which represents a uniform random distribution, rather than the correct one, `np.random.randn`. After submission, I corrected the codes and rerun the agent. Now the noise is not biased anymore, as shown below

![Plot of OU Noise loss](./images/noise_after.png)

Another error occurs at the procedure of decaying the noise. Instead of multipling by a constant amount, I was subtracting one. Of course, as steps go to infinity, the final noise will become large negative value. Fortunately, since the steps were not large enough, `Epsilon` was never reduced to close to zero. The new decaying process is shown below, at the end of the training it reaches 40% of the starting value:

![Plot of Epsilon Decay](./images/epsilon.png)

### Critic Learning Rate Tuning

Last but not least, I mention here the tuning process of the learning rate. To increase the speed of learning, I adjust both of the learning rate to 2e-3, which leads to a better agent than initial one. However, still it was not enough to reach the target benchmark. I started to examine various losses during training using Tensorboard. 

![Plot of critic loss](./images/critic_loss.png)

One can see that after some initial episodes, the critic loss function started to oscillate dramatically. This is a sign of too large learning rate, thus, I tuned down the critic learning rate. The final learning rate was tuned to be:
```python 
LR_CRITIC = 1e-4 
```
After that, the agent performs much better and also reduces the actor loss function as can be seen below. The yellow line which represents actor losses after tuning can reach a lower level than before.

![Plot of actor loss](./images/actor_loss.png)


## Model Results
The final pytorch model results for actor and critic are stored in the following two files.

   - [Actor Nework](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/checkpoint_actor.pth)
   - [Critic Nework](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/checkpoint_critic.pth)


# Plot of Rewards

Given the chosen architecture and parameters, our results are shown in this rewards curve
![Plot of Rewards](./images/reward_curve.png)

The problem can be considered solved at around 100 iteration. Also the agent turned out to be extremely stable, which can be seen for 400 episodes. As seen from Tensorboard record, the final model took 3.5 hours to train for 500 episodes on a Nvidia GTX 1660 GPU with CPU of Intel® Core™ i5-4460 CPU @ 3.20GHz × 4. The RAM spec is 16GB.  
![Plot of Time to Train](./images/time_training.png)

##### Rewards of 2nd Submission

Fortunately, the traning progress did not change much after the correction of the noise term. The score process is almost the same as last iteration.

![Plot of Score 2nd](./images/score_after.png)



# Ideas for Future Work

### *PPO Algorithm*
Apart from DDPG, more advanced policy based methods such as [Proximal Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347) maybe beneficial for such training. 

I started to code the agent using a vanilla PPO agent adapted from Udacity's PPO class. The [`ppo_model.py`](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ppo_model.py) describes my PPO policy network structure, which is similar to the actor network. The notebook [`Continuous_Control_PPO.ipynb`](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/Continuous_Control_PPO.ipynb) records my attempt to use the PPO agent to solve the Reacher problem. However, as indicated in the Cell [39] of the notebook, the performance of the agent is far from satisfactory. This is a major opportunity that requires improvement. My understanding is that a lot of hyper-parameters still need to be tuned away from the vanilla agent. That requires significant work, but I am confident that a good performance could be achieved finally.


### *Experiment with other algorithms*
Tuning the DDPG or PPO algorithms required a lot of trial and error. Perhaps another algorithm such as [Trust Region Policy Optimization (TRPO)](https://arxiv.org/abs/1502.05477) or [Distributed Distributional Deterministic Policy Gradients (D4PG)](https://arxiv.org/abs/1804.08617) would be more robust.


###  Add *gradient clipping*  or *batch norm*
I've used both gradient clipping and batch normalization many times in the past when building CNN/RNN, in order to prevent gradient from exploding. But, it didn't occur to me how important it would be to this project. I think it would be beneficial but I still need to experiment with those technique to see how much.

# Ideas for Future Work

### *PPO Algorithm*
Apart from DDPG, more advanced policy based methods such as [Proximal Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347) maybe beneficial for such training. 

I started to code the agent using a vanilla PPO agent adapted from Udacity's PPO class. The [`ppo_model.py`](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/ppo_model.py) describes my PPO policy network structure, which is similar to the actor network. The notebook [`Continuous_Control_PPO.ipynb`](https://github.com/readerwei/Reinforcement_Learning_Degree/blob/master/p2_continuous-control/Continuous_Control_PPO.ipynb) records my attempt to use the PPO agent to solve the Reacher problem. However, as indicated in the Cell [39] of the notebook, the performance of the agent is far from satisfactory. This is a major opportunity that requires improvement. My understanding is that a lot of hyper-parameters still need to be tuned away from the vanilla agent. That requires significant work, but I am confident that a good performance could be achieved finally.


### *Experiment with other algorithms*
Tuning the DDPG or PPO algorithms required a lot of trial and error. Perhaps another algorithm such as [Trust Region Policy Optimization (TRPO)](https://arxiv.org/abs/1502.05477) or [Distributed Distributional Deterministic Policy Gradients (D4PG)](https://arxiv.org/abs/1804.08617) would be more robust.


###  Add *gradient clipping*  or *batch norm*
I've used both gradient clipping and batch normalization many times in the past when building CNN/RNN, in order to prevent gradient from exploding. But, it didn't occur to me how important it would be to this project. I think it would be beneficial but I still need to experiment with those technique to see how much.